# Setting the locale of a notebook for Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/enabling-languages/python-i18n/blob/main/colab/locale_module_colab.ipynb)

## Overview

With Google Colab, it is necessary to distinguish between the locale used by the Colab user interface, and the locale used by python when executing code. The Colab user interface uses the language settings in your web browser. The Python environment uses the locale of the initialised runtime environment the notebook is running in.

## Configuring Google Colab

When a runtime is initialised there is limited locale support. The available locales are:

* `C`
* `C.UTF-8`
* `en_US.utf8`
* `POSIX`

The runtime environment will be using `en_US.utf8`. If you require your notebook to use a different locale, it is necessary to use the `locale` module at the beginning of the notebook. The locale should not be subsequently changed. If you require multiple locales during code execution, or need code to be flexible about the locale being used, it is best to use `PyICU`.

To use a different locale in a notebook, it is necesary to:

1. install the necessary language packs, and
2. restart the runtime environment.

The notebook contains example code that will try to set the required locale, and if the locale is unavailable, the code will install the language pack required for this notebook, and then restrt the runtime environment.

It will be necessary to manually rerun the notebook.

In [ ]:
import platform
platform.platform()

When this notebook was authoured, the Colab runtime environment was based on Ubuntu 18.04 LTS (Bionic).

Refer to the [list of lanaguage packs](https://packages.ubuntu.com/bionic/translations/) available in the Ubuntu Package archive.

In [ ]:
# List all current 
!locale -a 

The file `/usr/share/i18n/SUPPORTED` lists all the supported locales that you can use with Google Colab.

In [ ]:
import re

# Generate a list of all UTF-8 locales supported in current Google Colab runtime
supported = "/usr/share/i18n/SUPPORTED"
supported_locales = []
with open(supported) as fp:
    for line in fp:
        l = line.split()
        if re.search(r'UTF\-8',l[0]):
            supported_locales.append(l[0])
        elif not re.search(r'UTF\-8', l[0]) and l[1] == "UTF-8":
            supported_locales.append((".").join(l))

# All supported locales in current Google Colab runtime
print(supported_locales)

In [ ]:
# Return list of UTF-8 locales that support specified language
def matching_locales(lang):
  r = re.compile(r'^'+lang+'_.+')
  return list(filter(r.match, supported_locales))

#lang = input("Language code: ")
lang = "fr"
print(matching_locales(lang))

To identify the default system locale usd by Python, use [locale.getdefaultlocale()](https://docs.python.org/3/library/locale.html#locale.getdefaultlocale).

In [ ]:
import locale
default_locale = locale.getdefaultlocale()
default_locale

There are two approaches to installing additional locales on Google Colab:

1. Install the appropriate language pack
2. Use `locale-gen` and `update-locale`

The second approach is probably best for Google Colab. The first will install syetm localisation files as well as the locales, while the second approach will just install the locales required.

## Using language packs

The following code cell, when running on Google Colab, will set the locale to English (Australia). If locale is unavailable, it will install the language pack required by the notebook then restart the Colab runtime environment. After the runtime environment is reset, it will be necessary to rerun the code in the notebook.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except ImportError:
  IN_COLAB = False

if IN_COLAB:
  try:
    locale.setlocale(locale.LC_ALL, "en_AU.UTF-8")
  except locale.Error:
    print("Currently installing required locale. After the install the runtime environment will restart, please rerun all code.")
    !sudo locale-gen en_AU.UTF-8
    !sudo update-locale
    import os
    os.kill(os.getpid(), 9)
else:
    try:
        locale.setlocale(locale.LC_ALL, "en_AU.UTF-8")
    except locale.Error:
        locale.setlocale(locale.LC_ALL, "")

In [ ]:
# Current locale
locale.getlocale()

## Using `locale-gen` and `update-locale`

Alternatively, it is possible tp use `locale-gen` and `update-locale` to install the necessary locale.

In [ ]:
#loc = input('Enter required locale: ')
loc = "am_ET.UTF-8"

# If locale is supported, set to locale or install locale and restsrt runtime.
# If locale is unsupported fallback to default locale.
if loc in supported_locales:
  try:
    locale.setlocale(locale.LC_ALL, loc)
  except locale.Error:
    print("After the missing locale is installed, the runtime environment will restart, please rerun all code.")
    !sudo locale-gen {loc}
    !sudo update-locale
    import os
    os.kill(os.getpid(), 9)
else:
  locale.setlocale(locale.LC_ALL, '')

print("Current locale: ", locale.getlocale())

## Notes

_When adding locales you are adding system resources. In order to make them available to Python it is necessary to restart the runtime environement and rerun the notebook code._

_The locale should be set near the beginning of the notebook, and should not be changed._

<div style="text-align: center; text-style: italic;">© 2021-2022 <a href="https://enabling-languages.github.io/">Enabling Languages</a>. <br>
Released under the <a href="https://github.com/enabling-languages/python-i18n/blob/main/LICENSE">MIT license</a>.</div>